# 1.Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import os
import zipfile
import json
import copy
import warnings
warnings.filterwarnings('ignore')

# 2.Import Datasets

In [ ]:
# personal directory

# with zipfile.ZipFile('./listings.zip', 'r') as zip_ref:
#   try:
#     zip_ref.extractall('./')
#     if 'listings.csv' not in os.listdir('./'):
#         raise Exception('Failed to unzip files')
#     else:
#       print('Unzipped files')
#   except:
#     print('Failed to unzip files')

Unzipped files


In [ ]:
# personal directory
# det_list_df = pd.read_csv('./listings.csv', dtype={'id':str})

# GitHub directory
# det_list_df = pd.read_csv('../data/raw/AirBnB/listings.csv', dtype={'id':str})
det_list_df = pd.read_csv('../data/raw/AirBnB/samples/listings.csv', dtype={'id':str})

In [ ]:
det_list_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6422,https://www.airbnb.com/rooms/6422,20221221170309,2022-12-22,city scrape,Nashville Charm,30 day or more rental during COVID. Show COVID...,Historic East Nashville is home to many new an...,https://a0.muscache.com/pictures/pro_photo_too...,12172,...,4.96,4.92,4.98,NaN,f,1,0,1,0,4.06
1,59576,https://www.airbnb.com/rooms/59576,20221221170309,2022-12-21,city scrape,Large Main Suite near Lake *ladies only NS plz,non-smokers only please<br /><br />furnished r...,We live on a cul-de-sac in a small community o...,https://a0.muscache.com/pictures/4e4e34db-3155...,812128,...,5.00,5.00,5.00,NaN,f,10,1,9,0,0.34
2,72906,https://www.airbnb.com/rooms/72906,20221221170309,2022-12-21,city scrape,Vandy/Belmont/10 mins to Broadway - Sunny 800 ...,Entire top floor. Private. 800 sq ft of bright...,Historic Belmont-Hillsboro neighborhood. Walk-...,https://a0.muscache.com/pictures/58602855/3788...,176117,...,4.99,4.96,4.89,NaN,f,1,1,0,0,4.76
3,319705,https://www.airbnb.com/rooms/319705,20221221170309,2022-12-21,city scrape,"SuperSweetSTUDIO, jacuzzi, open Nov 23, 6 mo",Huge fully furnished private room /studio apar...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/3470453/c7d82...,22296,...,4.68,4.62,4.57,NaN,f,8,0,7,1,0.36
4,289242,https://www.airbnb.com/rooms/289242,20221221170309,2022-12-21,city scrape,"MorningstarHouse, monthly room- open Aug 19",Morningstar House! Extended stay - price reduc...,The Morningstar House is in a quiet cul-de-sac...,https://a0.muscache.com/pictures/40a7ab72-20f7...,22296,...,4.78,4.47,4.69,NaN,f,8,0,7,1,0.57


In [ ]:
det_list_df.shape

(8127, 75)

## 2.1 Filter Against Already Processed AirBnB Listings

In [ ]:
# personal directory

# for file in os.listdir(os.getcwd()):
#   if re.match('AirBnB Processed\s\d{4}_\d{2}_\d{2}.zip', file):
#     with zipfile.ZipFile(file, 'r') as zip_ref:
#       try:
#         zip_ref.extractall('./')
#         print('Unzipped: {}'.format(file))
#       except:
#         print('Failed to unzip files')

Unzipped: AirBnB Processed 2023_03_23.zip
Unzipped: AirBnB Processed 2023_03_26.zip
Unzipped: AirBnB Processed 2023_03_25.zip


In [ ]:
# Github directory

for file in os.listdir('../data/interim/AirBnB Nearby Entertainment Processed/'):
  if re.match('AirBnB Processed\s\d{4}_\d{2}_\d{2}.zip', file):
    with zipfile.ZipFile('../data/interim/AirBnB Nearby Entertainment Processed/' + file, 'r') as zip_ref:
      try:
        zip_ref.extractall('../data/interim/AirBnB Nearby Entertainment Processed/')
        print('Unzipped: {}'.format(file))
      except:
        print('Failed to unzip files')

In [ ]:
# personal directory

# processed_listings = pd.DataFrame()

# for file in os.listdir(os.getcwd()):
#   if re.match('^AirBnB Processed\s\d{4}_\d{2}_\d{2}$', file): # alternative check
#   if os.path.isdir(file) and re.match('AirBnB Processed\s\d{4}_\d{2}_\d{2}', file):
#     for sub_file in os.listdir(os.getcwd()+'/'+file):
#       if re.match('processed_airbnb_batch_\d_\d{4}_\d{2}_\d{2}\s\d{2}_\d{2}_\d{2}.csv', sub_file):
#         print(sub_file)
#         found_file = re.findall('processed_airbnb_batch_\d_\d{4}_\d{2}_\d{2}\s\d{2}_\d{2}_\d{2}.csv', sub_file)[0]
#         cached_listings = pd.read_csv(os.getcwd()+'/'+file+'/'+sub_file, dtype={'AirBnB_id':str})
#         if processed_listings.empty:
#           processed_listings = cached_listings
#         else:
#           processed_listings = pd.concat([processed_listings, cached_listings])

processed_airbnb_batch_0_2023_03_25 19_55_18.csv
processed_airbnb_batch_4_2023_03_25 19_55_19.csv
processed_airbnb_batch_2_2023_03_25 19_55_18.csv
processed_airbnb_batch_1_2023_03_25 19_55_18.csv
processed_airbnb_batch_5_2023_03_25 19_55_19.csv
processed_airbnb_batch_3_2023_03_25 19_55_19.csv
processed_airbnb_batch_1_2023_03_26 15_56_56.csv
processed_airbnb_batch_5_2023_03_26 15_56_57.csv
processed_airbnb_batch_2_2023_03_26 15_56_56.csv
processed_airbnb_batch_3_2023_03_26 15_56_57.csv
processed_airbnb_batch_0_2023_03_26 15_56_56.csv
processed_airbnb_batch_4_2023_03_26 15_56_57.csv
processed_airbnb_batch_2_2023_03_23 19_43_00.csv
processed_airbnb_batch_1_2023_03_23 19_43_00.csv


In [ ]:
# Github directory

processed_listings = pd.DataFrame()

for file in os.listdir('../data/interim/AirBnB Nearby Entertainment Processed/'):
  # if re.match('^AirBnB Processed\s\d{4}_\d{2}_\d{2}$', file): # alternative check
  if os.path.isdir(file) and re.match('AirBnB Processed\s\d{4}_\d{2}_\d{2}', file):
    for sub_file in os.listdir('../data/interim/AirBnB Nearby Entertainment Processed/' + file):
      if re.match('processed_airbnb_batch_\d_\d{4}_\d{2}_\d{2}\s\d{2}_\d{2}_\d{2}.csv', sub_file):
        print(sub_file)
        found_file = re.findall('processed_airbnb_batch_\d_\d{4}_\d{2}_\d{2}\s\d{2}_\d{2}_\d{2}.csv', sub_file)[0]
        cached_listings = pd.read_csv('../data/interim/AirBnB Nearby Entertainment Processed/' + file + '/'+sub_file, dtype={'AirBnB_id':str})
        if processed_listings.empty:
          processed_listings = cached_listings
        else:
          processed_listings = pd.concat([processed_listings, cached_listings])

In [ ]:
processed_listings['AirBnB_id'].nunique()

6738

In [ ]:
det_list_df = det_list_df[~det_list_df['id'].isin(processed_listings['AirBnB_id'].values)]

In [ ]:
det_list_df.shape

(1389, 75)

# 3.Create API Connection to Yelp

In [ ]:
# personal directory
# with open('./Yelp API Key.txt', 'r') as reader:

# GitHub directory
with open('../Yelp API Key.txt', 'r') as reader:
  api_key_header = reader.readline()

if re.search("^API Key : ", api_key_header):
  print('Match')
  api_key = re.findall("(?<=API Key : ).+", api_key_header)[0]
else:
  print('Check .txt File')

# 4.Call Yelp API

In [ ]:
# https://docs.developer.yelp.com/reference/v3_business_search

import requests
from urllib.error import HTTPError

def yelp_api_call(key, lat, long, rad):

  # create call string
  url = "https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&radius={}&categories=active&categories=arts&categories=food&categories=nightlife&categories=restaurants&categories=shopping&sort_by=best_match&limit=20".format(str(lat), str(long), str(rad))

  # create HTTP Header object 
  headers = {
      "accept": "application/json",
      "Authorization": "Bearer {}".format(key)
  }

  # call Yelp 
  response = requests.get(url, headers=headers)

  return response


In [ ]:
det_list_df_copy = det_list_df.copy(deep=True)

In [ ]:
det_list_df_copy.shape

(1389, 75)

In [ ]:
import time

def get_feed(df, upper_lim=5, radius=11265):
  # (11265 meters ~ 7 miles)

  business_df = pd.DataFrame()

  counter = 0
  for index, row in df.iterrows():
    if counter<upper_lim:
      r = yelp_api_call(api_key, row['latitude'], row['longitude'], radius)

      # https://www.w3schools.com/python/ref_requests_response.asp
      if r.status_code==200:
        temp_df = pd.json_normalize(r.json()['businesses'])
        temp_df['AirBnB_id'] = row['id']
        # print(row['id'])
        # print(temp_df.shape)

        if business_df.empty:
            business_df = temp_df
        else:
          business_df = pd.concat([business_df, temp_df])  

      else:
        print(r.text)
        break

      if counter%500==0:
        time.sleep(3)
        
      counter+=1

  return business_df.reset_index()

In [ ]:
%%time
yelp_feed_df = get_feed(det_list_df_copy, 3000)

CPU times: user 44.5 s, sys: 1.63 s, total: 46.2 s
Wall time: 9min 3s


In [ ]:
yelp_feed_df_copy = yelp_feed_df.copy(deep=True)

In [ ]:
yelp_feed_df_copy['AirBnB_id'].nunique()

1389

# 5.Export Results and Manually Zip Batch Files

In [ ]:
# https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks

def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  for i in range(0, len(lst), n):
      yield lst[i:i + n]

In [ ]:
for chunk in chunks(range(0, 77), 10):
  print(chunk)
  print('start, end: {}, {}\n'.format(chunk[0], chunk[-1]))

range(0, 10)
start, end: 0, 9

range(10, 20)
start, end: 10, 19

range(20, 30)
start, end: 20, 29

range(30, 40)
start, end: 30, 39

range(40, 50)
start, end: 40, 49

range(50, 60)
start, end: 50, 59

range(60, 70)
start, end: 60, 69

range(70, 77)
start, end: 70, 76



In [ ]:
from datetime import datetime

def yield_feed(yelpfeed, batch_sz=10000):
  curr_batch=0

  # Step into the yelpfeed
  feed_len = yelpfeed.shape[0]

  for chunk in chunks(range(0, feed_len), batch_sz):
    yielded_df = yelpfeed.iloc[chunk[0]:chunk[-1]+1]

    today_date = datetime.today().strftime('%Y_%m_%d')
    today_date_time = datetime.today().strftime('%Y_%m_%d %H:%M:%S')

    # save scraped nearby entertainment from this session
    # yielded_df.to_csv('./Yelp Fetched {}/yelp_fetched_batch_{}_{}.csv'.format(today_date, curr_batch, today_date_time), index=False) # personal directory

    yielded_df.to_csv('../data/raw/Yelp Fetched/yelp_fetched_batch_{}_{}.csv'.format(curr_batch, today_date_time), index=False) # GitHub directory

    # save scraped AirBnB listings from this session
    processed_airbnb = yielded_df['AirBnB_id'].unique()
    processed_airbnb_df = pd.DataFrame(processed_airbnb, columns=['AirBnB_id'])
    # processed_airbnb_df.to_csv('./AirBnB Processed {}/processed_airbnb_batch_{}_{}.csv'.format(today_date, curr_batch, today_date_time), index=False) # personal directory
    processed_airbnb_df.to_csv('../data/interim/AirBnB Nearby Entertainment Processed/processed_airbnb_batch_{}_{}.csv'.format(curr_batch, today_date_time), index=False) # GitHub directory


    curr_batch+=1


In [ ]:
yield_feed(yelp_feed_df_copy, 10000)

27780
